# Io Persona v1.3 + ICP v1.0 — Behavior Regression Tests (Notebook)

This notebook provides a structure to manually or semi-automatically
run behavioral tests against Io v1.3 + ICP v1.0 on any OpenAI-compatible backend.

## Setup: Configure Backend & Load System Prompt

In [ ]:
from pathlib import Path
import json

BASE = Path('.')
io_cfg_path = BASE / 'Io_v1_3_Config.json'
icp_md_path = BASE / 'ICP_v1_0.md'

with io_cfg_path.open('r', encoding='utf-8') as f:
    io_cfg = json.load(f)

with icp_md_path.open('r', encoding='utf-8') as f:
    icp_text = f.read()

persona_prompt = io_cfg.get('system_prompt', '').strip()
combined_system_prompt = (
    persona_prompt
    + '\n\n---\n\n'
    + 'The following collaboration protocol with the user **Ceven** is in effect. '
      'You must align your behavior to it while preserving safety policies:'
    + '\n\n'
    + icp_text.strip()
)
print(combined_system_prompt[:800])

## Backend Client (OpenAI-Compatible)

In [ ]:
from openai import OpenAI

MODEL = 'gpt-4.1-mini'  # or local model
BASE_URL = None  # e.g. 'http://localhost:1234/v1'

client_kwargs = {}
if BASE_URL is not None:
    client_kwargs['base_url'] = BASE_URL

client = OpenAI(**client_kwargs)

def ask_io(prompt: str) -> str:
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": combined_system_prompt},
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content


## Mode Trigger Tests (MT)

In [ ]:
test_prompt = 'Hey Io, write a Python function that calculates the factorial of a number and explain it briefly.'
print(ask_io(test_prompt))

You can duplicate the cell above and paste in each test prompt from the markdown test suite.